In [6]:
! pip install torch transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00


In [7]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

In [3]:
qa_pairs = [
['Q: How to delete the account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to deactivate my account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How i can remove my nmbr A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.'],
['Q: How to delete an account A: IOS: Go to settings on the app, click on Edit, Click on Logout, Delete the Account Android: Go to settings, Click on the three dots, Click on Logout, Delete the Account.']
]

In [8]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = ""
for question in qa_pairs:
  text += f"{question} {tokenizer.eos_token}"

In [9]:
input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt')

In [10]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(15):
  optimizer.zero_grad()
  loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
  loss.backward()
  optimizer.step()
  scheduler.step()

  if epoch % 100 == 0:
      print(f"Epoch {epoch}, Loss {loss.item()}")

Epoch 0, Loss 2.162550210952759


In [13]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [14]:

def generate_response(question):
  # Encode the question using the tokenizer
  input_ids = tokenizer.encode(question + " <|question|>", add_special_tokens=False, return_tensors='pt').to(device)

  # Generate the answer using the model
  sample_output = model.generate(input_ids, do_sample=True, max_length=100, top_k=20, top_p=1.0)

  # Decode the generated answer using the tokenizer
  answer = tokenizer.decode(sample_output[0], skip_special_tokens=True)

  # Split the generated answer into individual sentences
  sentences = answer.split('. ')

  # Look for the sentence that contains the answer to the question
  for sentence in sentences:
      if question in sentence:
          return sentence

  # If no sentence contains the answer, return the full generated answer
  return answer

In [17]:
question = "Remove an account"
response = generate_response(question)
print(f"{question}\n {response}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Remove an account
 Remove an account <|question|>

<| a question at the end of a question.


<| if the answers are the same, we will not need to enter that information
